In [1]:
import sys
import os
import random

import numpy as np
import torch
import transformers
import datasets


In [2]:
tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-uncased", cache_dir='../bert_base_cache')
model = transformers.AutoModelForMaskedLM.from_pretrained("bert-base-uncased", cache_dir='../bert_base_cache')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
semeval_laptop = datasets.load_dataset(
    '../dataset_scripts/semeval2014_task4/semeval2014_task4.py',
    data_files={
        'train': '../dataset_files/semeval_2014/Laptops_Test_Gold.xml',
        'test': '../dataset_files/semeval_2014/Laptop_Train_v2.xml'
    },
    cache_dir='../dataset_cache')

semeval_laptop = semeval_laptop["test"].select(range(50))
print(semeval_laptop[0])

Using custom data configuration default-b5c3bb350489de8a
Reusing dataset sem_eval2014_task4_dataset (../dataset_cache\sem_eval2014_task4_dataset\default-b5c3bb350489de8a\0.0.1\f33ba7108331ad17be3f9fb710ca001edb383fba797c6ed0938354e6812ca969)


{'aspect': 'cord', 'sentiment': 2, 'text': 'I charge it at night and skip taking the cord with me because of the good battery life.'}


In [4]:
sentiment_prompts = [
    "The {aspect} is [MASK].",
    "I [MASK] the {aspect}."
]

In [5]:
def add_prompts(review, prompts):
    
    inputs = []
    sentiments = []
    for p in prompts:
        aspect_prompt = p.format(aspect=review["aspect"])
        review_and_prompt = review["text"] + " " + aspect_prompt
        inputs.append(review_and_prompt)
        sentiments.append(review["sentiment"])

    return {"input":inputs, "label": sentiments}

In [6]:
prompt_dataset = semeval_laptop.map(
    lambda e: add_prompts(e, sentiment_prompts),
    remove_columns=semeval_laptop.column_names)

Loading cached processed dataset at ../dataset_cache\sem_eval2014_task4_dataset\default-b5c3bb350489de8a\0.0.1\f33ba7108331ad17be3f9fb710ca001edb383fba797c6ed0938354e6812ca969\cache-a449803e4c20fe1d.arrow


In [7]:
def tokenize_helper(review_prompts, tokenizer, model):
    tokens = tokenizer(review_prompts["input"], truncation=True, padding='max_length', max_length=256, return_tensors="pt")
    outputs = model(**tokens)
    return {"outputs":outputs["logits"].tolist(), "label": review_prompts["label"]}


token_dataset = prompt_dataset.map(
    lambda e: tokenize_helper(e, tokenizer, model),
    batched=True, batch_size = 8, num_proc=8)

NameError: name 'tokenize_helper' is not defined

In [ ]:
model_output